In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rf-model/lgbm_model.pkl
/kaggle/input/logits-roberta-deberta-final/roberta_val_without_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_train_without_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_test_without_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_test_with_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_train_with_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_test_without_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_val_without_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_val_with_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_train_without_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_test_with_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_train_with_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_val_with_para.npy
/kaggle/input/logits-roberta-deberta-final/wandb/run-20240425_083609-kegt07fm/run-kegt07fm.wandb
/kaggle

In [50]:
! pip install accelarate -U
! pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate


In [51]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)
wandb.init()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [52]:
from sklearnex import patch_sklearn
patch_sklearn()


import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import matplotlib.pyplot as plt

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

PATH="/kaggle/input/webis-clickbait-22/"
PATH_LOGITS="/kaggle/input/logits-roberta-deberta-final/"
OUTPATH="/kaggle/working/"
# PATH = "D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH = "D:\\ghd\\NLP-Project\\output\\"
BATCH=8
sep='[SEP]'

label_encoder = LabelEncoder()

roberta = "roberta-base"
deberta = "microsoft/deberta-base"

Using CPU


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [53]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class Dataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels=None):
        self.labels = labels
        self.encodings = encodings
    
    def __getitem__(self,idx):
        item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
        if self.labels is not None:
            item["label"]  = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [54]:
data_train=pd.read_json("/kaggle/input/webis-clickbait-22/train.jsonl",lines=True,encoding='utf-8')
data_test=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True,encoding='utf-8')
data_val=pd.read_json("/kaggle/input/webis-clickbait-22/validation.jsonl",lines=True,encoding='utf-8')


In [55]:
label={
    'multi':0,
    'passage':1,
    'phrase':2
}

In [56]:
def get_split_data(data,if_train,option):
    global label_encoder
#     data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        if option==1:
            i['text']=i['postText']+" "+i['targetTitle']+" "+i['targetKeywords']+" "+sep+i['targetDescription']
        else:
            i['text']=i['targetParagraphs']+sep+i['postText']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y

In [57]:
x_train_without_para,y_train = get_split_data(data_train,True,1)
x_test_without_para,y_test = get_split_data(data_test,False,1)
x_val_without_para,y_val= get_split_data(data_val,False,1)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [58]:
x_train_without_para,y_train = get_split_data(data_train,True,1)
x_test_without_para,y_test = get_split_data(data_test,False,1)
x_val_without_para,y_val= get_split_data(data_val,False,1)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [59]:
logits_train_roberta_without_para = np.load(PATH_LOGITS+"roberta_train_without_para.npy")
logits_train_roberta_with_para = np.load(PATH_LOGITS+"roberta_train_with_para.npy")
logits_train_deberta_without_para = np.load(PATH_LOGITS+"deberta_train_without_para.npy")
logits_train_deberta_with_para = np.load(PATH_LOGITS+"deberta_train_with_para.npy")

In [60]:
logits_val_roberta_without_para = np.load(PATH_LOGITS+"roberta_val_without_para.npy")
logits_val_roberta_with_para = np.load(PATH_LOGITS+"roberta_val_with_para.npy")
logits_val_deberta_without_para = np.load(PATH_LOGITS+"deberta_val_without_para.npy")
logits_val_deberta_with_para = np.load(PATH_LOGITS+"deberta_val_with_para.npy")

In [61]:
logits_test_roberta_without_para = np.load(PATH_LOGITS+"roberta_test_without_para.npy")
logits_test_roberta_with_para = np.load(PATH_LOGITS+"roberta_test_with_para.npy")
logits_test_deberta_without_para = np.load(PATH_LOGITS+"deberta_test_without_para.npy")
logits_test_deberta_with_para = np.load(PATH_LOGITS+"deberta_test_with_para.npy")

In [62]:
combined_train = np.concatenate([logits_train_roberta_without_para,logits_train_roberta_with_para,logits_train_deberta_without_para,logits_train_deberta_with_para],axis=2)
combined_val = np.concatenate([logits_val_roberta_without_para,logits_val_roberta_with_para,logits_val_deberta_without_para,logits_val_deberta_with_para],axis=2)
combined_test = np.concatenate([logits_test_roberta_without_para,logits_test_roberta_with_para,logits_test_deberta_without_para,logits_test_deberta_with_para],axis=2)

In [63]:
combined_train = np.squeeze(combined_train,axis=1)
combined_val = np.squeeze(combined_val,axis=1)
combined_test = np.squeeze(combined_test,axis=1)

In [64]:
standard_scaler = StandardScaler()
combined_train = standard_scaler.fit_transform(combined_train)
combined_val = standard_scaler.transform(combined_val)
combined_test = standard_scaler.transform(combined_test)

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [65]:
# param_grid = {
#     'n_estimators':[10, 20, 50, 100, 250, 500],
#     'max_depth': [5, 7, 9, 11,13,15],
#     'min_samples_split': [2, 5, 10, 15, 20],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, verbose=0, scoring='balanced_accuracy', n_jobs=-1)
# grid.fit(combined_train, y_train)

# grid.best_params_

In [66]:
# max_depth = grid.best_params_['max_depth']
# min_samples_leaf = grid.best_params_['min_samples_leaf']
# n_estimators = grid.best_params_['n_estimators']
# min_samples_split = grid.best_params_['min_samples_split']
# bootstrap = grid.best_params_['bootstrap']

In [67]:
# lgb_train = lgb.Dataset(combined_train, label=y_train)
# lgb_val = lgb.Dataset(combined_val, label=y_val)

# model = RandomForestClassifier(
#     max_depth=max_depth,
#     min_samples_leaf=min_samples_leaf,
#     n_estimators=n_estimators,
#     min_samples_split=min_samples_split,
#     bootstrap=bootstrap,
#     random_state=42
# )
# model.fit(combined_train, y_train)

In [68]:
# y_pred_train=model.predict(combined_train)
# y_pred_val=model.predict(combined_val)

# print("Training Metrics")
# print(balanced_accuracy_score(y_train,y_pred_train))
# print(f1_score(y_train,y_pred_train,average='macro'))
# print(classification_report(y_train,y_pred_train))

# print("Validation Metrics")
# print(balanced_accuracy_score(y_val,y_pred_val))
# print(f1_score(y_val,y_pred_val,average='macro'))
# print(classification_report(y_val,y_pred_val))


In [69]:
# lgb.plot_metric(model)

In [70]:
# import pickle
# with open("lgbm_model.pkl","wb") as f:
#     pickle.dump(model,f)

In [71]:
with open("/kaggle/input/rf-model/lgbm_model.pkl","rb") as f:
    loaded_model = pickle.load(f)

In [72]:
y_pred = loaded_model.predict(combined_test)
print('Testing accuracy {:.4f}'.format(accuracy_score(y_test, y_pred)))
print("Testing data metrics:")
print(balanced_accuracy_score(y_test,y_pred))
print(f1_score(y_test,y_pred,  average='macro'))
print(classification_report(y_test,y_pred))

INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: running accelerated version on CPU


Testing accuracy 0.7250
Testing data metrics:
0.6962080879909199
0.7061989663812241
              precision    recall  f1-score   support

           0       0.71      0.57      0.63       174
           1       0.72      0.77      0.74       403
           2       0.74      0.74      0.74       423

    accuracy                           0.73      1000
   macro avg       0.72      0.70      0.71      1000
weighted avg       0.72      0.72      0.72      1000

